In [28]:
import numpy as np
import pandas as pd
import operator
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
import csv
import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef
import os
import string

In [29]:
#####################################################
################ Initialization #####################
#####################################################

#ground truth metadata file
labeledFile = 'ebu3b_sensor_types_location.csv'
with open(labeledFile, 'rb') as fp:
    #Import list of points to a data frame
    #Pandas data frame allows you to store records and manipulate them: http://pandas.pydata.org/
    points_list = pd.DataFrame.from_csv(fp)
    points_list = points_list.set_index(keys='Unique Identifier')
    points_list.index.str.strip()
    points_list = points_list.replace(r'\s+|-', '_', regex=True)
    
#Import lookup table that maps EBU3B descriptions to Brick tagsets
desc_tagset_map = pd.read_csv('ebu3b_description_to_tagset.csv')
desc_tagset_map = desc_tagset_map.replace(r'\s+','_',regex=True).set_index('BACnet Description').dropna()

In [30]:
#####################################################
######## Initialize RDFlib graph and namespaces #####
#####################################################

#Namespaces
#Note: The URLs are fake
BRICK = rdflib.Namespace('http://buildsys.org/ontologies/Brick#')
EBU3B = rdflib.Namespace('http://ucsd.edu/building/ontology/ebu3b#')
#RDF, RDFS and OWL have already been imported in the library initializations

#Initiate graph from base ttl file
building_graph = rdflib.Graph()
brick_graph = rdflib.Graph()
building_graph.bind('ebu3b', EBU3B)
building_graph.bind('brick', BRICK)
building_graph.bind('owl', OWL)
brick_graph.bind('brick', BRICK)
brick_graph.parse('../../BuildingSchema/Brick.ttl', format='turtle')
#building_graph.parse('../../BuildingSchema/Brick_Protege.owl', format='turtle')
#brick_graph.serialize(destination='brick_rdflib.ttl', format='turtle')

<Graph identifier=N5d94184c25a24775930347610287944e (<class 'rdflib.graph.Graph'>)>

In [31]:
#Load all equipment to building graph
#Get equipment related terms
equip_list = points_list[['Building','Equipment Type','Equipment Ref','Location']]
#Create equipment label
equip_list['equip_id'] = equip_list['Building'] + '_' + equip_list['Equipment Type'] + '_' + \
                            equip_list['Equipment Ref'] + '_' + equip_list['Location']
    
#Drop duplicates to get list of equipment only
equip_list = equip_list.set_index('equip_id').drop_duplicates()

#Iterate through list and add to schema
for equip_label, equip_metadata in equip_list.iterrows():
    #Add EBU3B prefix
    equip_label = EBU3B[equip_label]
    #Add to schema
    building_graph.add((equip_label, RDF.type, OWL.NamedIndividual))
    
    #Add equipment type
    equip_type = str(equip_metadata['Equipment Type'])
    if equip_type == 'nan': #skip blanks
        continue
    if 'CRAC' in equip_type: #CRAC formatting
        equip_type = 'CRAC'
    equip_type = BRICK[equip_type]
    if (equip_type, None, None) in brick_graph:
        #Add equip type to schema
        building_graph.add((equip_label, RDF.type, equip_type))
    else:
        print "Could not identify equipment type in schema:", equip_type
        building_graph.add((equip_label, RDF.type, BRICK.Equipment))
        
    

/Users/bbalaji/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
#Load all points to the building instance
for point_id, point_metadata in points_list.iterrows():
    #Create point label
    point_label = re.sub('\.+','_',point_metadata['Johnson Controls Name'])
    #Add building prefix
    point_label = EBU3B[point_label]
    #Add to Schema
    building_graph.add((point_label, RDF.type, OWL.NamedIndividual))

    #Adding Point Type
    #Look up the schema tagset
    desc = point_metadata['BACnet Description']
    if desc in desc_tagset_map.index and str(desc) != 'nan':
        point_type = desc_tagset_map.loc[desc]['Schema Label']
    else:
        continue
    if isinstance(point_type, pd.Series):
        point_type = point_type[0]
    if str(point_type) == 'nan':
        continue
    #Get the first match, capitalize the first letter of each word and concatenate by '_'
    #print point_type
    point_type = '_'.join([word[0].upper() + word[1:] for word in point_type.split('_')])
    #Take care of Synonyms
    if 'Supply Air' in point_type:
        point_type = point_type.replace('Supply Air','Discharge Air')
    #Concatenate with brick prefix
    point_type = BRICK[point_type]
    #print point_type
    #Check if point type exists in the schema
    if (point_type, None, None) in brick_graph:
        #Add point type to schema
        building_graph.add((point_label, RDF.type, point_type))
    else:
        print "Could not identify point type in schema:", point_type 
        building_graph.add((point_label, RDF.type, BRICK.Point))
    

Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://builds

In [33]:
# Store output as .ttl
building_graph.serialize(destination='ebu3b_brick.ttl', format='turtle')
# Search for "EBU3B" in the output file, you'll find the instances we created.

In [ ]:
#####################################################
######## Add triples and serialize them #############
#####################################################

#### Example for Point ####

###Adding Label####
#Read the point metadata by its unique identifier
point = points_list.loc['505_0_3001627']
#Create a label for the point
point_label = re.sub('\.+','_',point['Johnson Controls Name'])
#Add building prefix
point_label = EBU3B[point_label]
#print point_label
#Add to Schema
building_graph.add((point_label, RDF.type, OWL.NamedIndividual))

####Adding Point Type###
#Look up the schema tagset
point_type = desc_tagset_map.loc[point['BACnet Description']]['Schema Label']
#Get the first match, capitalize the first letter of each word and concatenate by '_'
point_type = '_'.join(point_type[0].title().split())
#Concatenate with brick prefix
point_type = BRICK[point_type]
#print point_type
#Check if point type exists in the schema
if (point_type, None, None) in brick_graph:
    #Add point type to schema
    building_graph.add((point_label, RDF.type, point_type))
else:
    print "Could not identify point type in schema:", point_type 

In [ ]:
###Example for a VAV instance ####
#Continuing with the point information above

##Create Equipment Label##
equip_label = point['Building'] + '_' + point['Equipment Type'] + '_' + point['Location']
#replaces spaces with '_'
equip_label = '_'.join(equip_label.split())
#add building prefix
equip_label = EBU3B[equip_label]
#print equip_label
#add to schema instance
building_graph.add((equip_label, RDF.type, OWL.NamedIndividual))

##Add equipment type##
equip_type = point['Equipment Type']
#add brick prefix
equip_type = BRICK[equip_type]
#print equip_type
#Check if equip type exists in the schema
if (equip_type, None, None) in brick_graph:
    #Add equip type to schema
    building_graph.add((equip_label, RDF.type, equip_type))
else:
    print "Could not identify equipment type in schema:", equip_type 
    
##Add relationship between point and equipment ##
building_graph.add((equip_label, BRICK.hasPoint, point_label))

In [ ]:
#TODO
#Iterate through all the points in the building and map them to schema
for point_id, point_metadata in points_list.iterrows():
    print point_metadata
    break

In [ ]:
#### For Graph Traversal #####
#For traversing the Brick or EBU3B graph, use the "Resource" part of RDFlib
#Tutorial here: http://rdflib.readthedocs.io/en/stable/apidocs/rdflib.html#module-rdflib.resource
#Example subclass traversal is shown below
valve = rdflib.resource.Resource(g, BRICK.Valve)
subclasses = list(valve.transitive_subjects(RDFS.subClassOf))
[c.qname() for c in subclasses]